In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import math
import cv2
import os

In [ ]:
def load_face_dataset(images_dir):
    faces = []
    names = []
    face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    
    for filename in os.listdir(images_dir):
        name = filename.split(".")[0]
        
        # load image
        image_path = os.path.join(images_dir, filename)
        image = cv2.imread(image_path)

        # convert image to gray scale
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # detect face position
        face_pos = face_classifier.detectMultiScale(image, minSize=(40, 40))
        for (x, y, w, h) in face_pos:
            # crop image to face size
            image = image[y:y+h, x:x+w]
            
        faces.append(image)
        names.append(name)
    
    # convert lists to NumPy arrays
    faces = np.array(faces, dtype="object")
    names = np.array(names)
    
    return faces, names

In [ ]:
def recognizer_LBPH(trainX, trainY):
    # train recognizer
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.train(trainX, trainY)
    return recognizer

In [ ]:
def prediction_LBPH(recognizer, testX, testY):
    predictions = []
    predicted_names = []
    confidence = []
    
    for face in testX:
        # get prediction and confidence
        predicted_name, confid = recognizer.predict(face)
        predictions.append(predicted_name)
        # convert integer prediction back to string
        predicted_name = le.inverse_transform([predicted_name])[0]
        predicted_names.append(predicted_name)
        confidence.append(confid)
    
    # show classification report
    print("Classification report:\n\n", classification_report(testY, predictions, target_names=le.classes_))
    return predicted_names, confidence

In [ ]:
def display_predictions(testX, testY, predicted_names, confidence):
    rows = math.ceil(len(testX) / 3)
    columns = 8
    fig = plt.figure(figsize=(16, 3*rows))

    i = 0
    count = 1
    for image in testX:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        actual_name = le.classes_[testY[i]]
        # plot image
        fig.add_subplot(rows, columns, count)
        plt.imshow(image)
        plt.axis("off")
        count += 1
        
        # add text
        fig.add_subplot(rows, columns, count)
        if predicted_names[i] == actual_name:
            plt.text(0, 0.6, "predicted: " + predicted_names[i], color="green")
        else:
            plt.text(0, 0.6, "predicted: " + predicted_names[i], color="red")
        plt.text(0, 0.5, "actual: " + actual_name)
        # lower confidence number, more confident prediction 
        plt.text(0, 0.4, "confidence: " + str(round(confidence[i], 2)))
        plt.axis("off")
        count += 1
        i += 1
    
    plt.show()

In [ ]:
images_dir = "C:\\Users\\cwqka\\Downloads\\EE4208 Intelligent Systems Design\\Assignment 1\\Images"

# load face dataset
faces, names = load_face_dataset(images_dir)
# encode the names as integers
le = LabelEncoder()
names_int = le.fit_transform(names)
# split dataset
(trainX, testX, trainY, testY) = train_test_split(faces, names_int, test_size=0.25, stratify=names_int, random_state=42)

# train LBPH model
recognizer = recognizer_LBPH(trainX, trainY)
# predict
predicted_names, confidence = prediction_LBPH(recognizer, testX, testY)
# display test images with predictions
display_predictions(testX, testY, predicted_names, confidence)